In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy import stats
import math
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures, StandardScaler
from sklearn import tree,linear_model,neighbors
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":10,"axes.titlesize":24,"axes.labelsize":24}) 

# Upload data

In [2]:
train = pd.read_csv('TrainClean.csv',index_col=0).reset_index(drop=True)
test = pd.read_csv('TestClean.csv',index_col=0).reset_index(drop=True)
NumOfSalesTra=train["NumberOfSales"]
NumOfCustomersTra=train["NumberOfCustomers"]
train.describe()

FileNotFoundError: File b'TrainClean.csv' does not exist

In [ ]:
train.columns

# Normalization

In [ ]:
all_data_index = train.columns.drop(['NumberOfSales','NumberOfCustomers'])
all_data = pd.concat((train.loc[:,all_data_index],test.loc[:,all_data_index]), ignore_index=True)
columns = all_data.columns

In [ ]:
x = all_data.values
min_max_scaler = MinMaxScaler()
all_data_normalized = pd.DataFrame(min_max_scaler.fit_transform(x))
all_data_normalized.columns = columns

In [ ]:
all_data_normalized

In [ ]:
test_offset = train.shape[0]
train_normalized = all_data_normalized.iloc[0:test_offset,:]
train_normalized['NumberOfSales'] = train["NumberOfSales"]
train_normalized['NumberOfCustomers'] = train["NumberOfCustomers"]
test_normalized = all_data_normalized.iloc[test_offset:all_data_normalized.shape[0],:]
train=train_normalized
test=test_normalized
np.isnan(train).sum()

# Lasso feature selection

In [ ]:
# Regression on NumberOfCustomers
y=train['NumberOfSales'].values.reshape(-1,1)
x=train.drop(columns=['NumberOfCustomers','NumberOfSales'],axis=1)
x.columns

In [ ]:
np.isnan(y).sum()

In [ ]:
lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=1
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    #print("%d\t%3.3f" % (i,c) )
    if abs(c)>threshold and not i == 0:
        selected.append(x.columns[i-1])
x=x[selected]
x.columns

In [ ]:
finaltrain=train[x.columns]
finaltrain["NumberOfSales"]=NumOfSalesTra
finaltrain["NumberOfCustomers"]=NumOfCustomersTra
finaltrain.to_csv("FinalTrain.csv")
finaltest=test[x.columns].to_csv("FinalTest.csv")